In [1]:
from langchain_ollama import OllamaLLM
from smolagents import LiteLLMModel

langchain_model = OllamaLLM(model="llama3.2")
smolagents_model = LiteLLMModel(
    model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)

c:\Users\Laís\Documents\Projetos\llm-agents\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dict_prompt = {
    'style': 'Pop',
    'user_prompt': 'A song about love and resilience',
    'tone': 'Sad'
}

In [3]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

def create_chain(parser_class, template, variables):
    parser = JsonOutputParser(pydantic_object=parser_class)
    prompt = PromptTemplate(
        template=template,
        input_variables=variables,
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | langchain_model | parser

    return chain

In [ ]:
class Lyrics(BaseModel):
    title: str = Field(description="Title of the song")
    intro: str = Field(description="Intro to the song")
    verse_one: str = Field(description="Verse one of the song")
    pre_chorus: str = Field(description="Pre-chorus of the song")
    chorus: str = Field(description="Chorus of the song")
    verse_two: str = Field(description="Verse two of the song")
    outro: str = Field(description="Outro of the song")


lyrics_template = '''You are a talented songwriter.
Consider a song with the following requisites:
- Style: {style}
- Prompt: {user_prompt}
- Tone: {tone}
Create the lyrics for this song, with the following sections: intro, verse one, pre-chorus, chorus, verse two and outro.
Ideally, create a strophe with at least 2 verses for each section. Finally, give it a title.
Answer in the following JSON format:
    - "title": title of the song
    - "intro": intro to the song
    - "verse_one": Verse one of the song
    - "pre_chorus": Pre-chorus of the song
    - "chorus": Chorus of the song
    - "verse_two": Verse two of the song
    - "outro": Outro of the song
'''

chain = create_chain(
    parser_class=Lyrics,
    template=lyrics_template,
    variables=['style', 'user_prompt', 'tone']
)

lyrics = chain.invoke(dict_prompt)

In [4]:
lyrics = {'title': 'Fading Away',
 'intro': ['In the silence, I hear your voice',
  'A whispered reminder of our choice',
  'To let go, to move on, to lose',
  'The love we had, the memories we chose'],
 'verse_one': ["We said forever, but forever's come and gone",
  "And now I'm left to face this empty dawn",
  'Your smile, a distant memory, a fleeting thought',
  "I'm searching for a way to heal my heart, to make it stop"],
 'pre_chorus': ['But the pain remains, like an open sore',
  'A constant reminder of what we had before',
  "I'm trying to move on, but I'm stuck in this place",
  "Where love and loss collide, and my heart can't escape"],
 'chorus': ["And I'm fading away, like a ghost in the night",
  "My heart is breaking, but it's not feeling right",
  "I'm trying to hold on, but you're slipping through my hands",
  'Leaving me with nothing but these tears and this pain'],
 'verse_two': ["We said forever, but forever's just a lie",
  "And now I'm left to wonder why",
  'You walked away, without a fight',
  'Leaving me here, to face the dark of night'],
 'outro': ["And I'll keep on fading, into the shadows of my mind",
  'Where love and loss entwine, and my heart will forever be left behind',
  "But maybe someday, I'll learn to let go",
  'And find my way back home, where love will set me free']}


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from typing import List, Dict
from smolagents import tool
import textstat

@tool
def analyze_verse_metrics(stanza: List[str]) -> List[Dict[str, int]]:
    """
    Counts the number of syllables in each verse of a stanza.

    Args:
        stanza: A list of verse lines.
    
    Returns:
        A list of dictionaries with the verse text and syllable count.
    """
    return [{"text": line.strip(), "syllables": textstat.syllable_count(line.strip())} for line in stanza]


@tool
def detect_rhyme_scheme(stanza: List[str]) -> List[str]:
    """
    Detects a basic rhyme scheme by comparing the last two letters of each verse.

    Args:
        stanza: A list of verse lines.
    
    Returns:
        A list representing the rhyme labels (A, B, C, ...).
    """
    suffixes = []
    rhyme_map = {}
    label = "A"
    
    for line in stanza:
        end = line.strip()[-2:].lower() if len(line.strip()) >= 2 else ""
        if end not in rhyme_map:
            rhyme_map[end] = label
            label = chr(ord(label) + 1)
        suffixes.append(rhyme_map[end])
    
    return suffixes


# @tool
# def detect_emotion(stanza: List[str]) -> str:
#     """
#     Roughly estimates the dominant emotion in the stanza using keywords.

#     Args:
#         stanza: A list of verse lines.
    
#     Returns:
#         A string representing a simple emotion label.
#     """
#     text = " ".join(stanza).lower()
#     if any(word in text for word in ["love", "happy", "shine", "sun"]):
#         return "joy"
#     elif any(word in text for word in ["cry", "dark", "alone", "pain"]):
#         return "sadness"
#     elif any(word in text for word in ["fight", "burn", "rage", "war"]):
#         return "anger"
#     else:
#         return "neutral"


@tool
def extract_keywords(stanza: List[str], top_k: int = 5) -> List[str]:
    """
    Extracts the most frequent keywords from the stanza.

    Args:
        stanza: A list of verse lines.
        top_k: Number of keywords to extract.

    Returns:
        A list of keywords.
    """
    text = " ".join(stanza)
    vectorizer = CountVectorizer(stop_words="english", max_features=top_k)
    X = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out().tolist()

In [ ]:
from smolagents import CodeAgent

agent = CodeAgent(
    tools=[analyze_verse_metrics, detect_rhyme_scheme, extract_keywords],
    model=smolagents_model
)

answer = agent.run(
    """
    You are an experienced songwriter. Your job is to analyze the lyrics of the following song: {song}
    Use all the tools you are provided with. Also, analyze the predominant emotion of each stanza. Repor your findings in a written summary.
    """.format(song=str(lyrics))
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an experienced songwriter. Your job is to analyze the lyrics of the following song: {'title': 'Fading   │
│ Away', 'intro': ['In the silence, I hear your voice', 'A whispered reminder of our choice', 'To let go, to move │
│ on, to lose', 'The love we had, the memories we chose'\], 'verse_one': ["We said forever, but forever's come    │
│ and gone", "And now I'm left to face this empty dawn", 'Your smile, a distant memory, a fleeting thought', "I'm │
│ searching for a way to heal my heart, to make it stop"\], 'pre_chorus': ['But the pain remains, like an open    │
│ sore', 'A constant reminder of what we had before', "I'm trying to move on, but I'm stuck in this place",       │
│ "Where love and loss collide, and my heart can't escape"\], 'chorus': ["And I'm fading away, like a ghost in    │
│ the night", "My heart is breaking, but it's not feeling right", "I'm trying to hold on, but you're slipping     │
│ through my hands", 'Leaving me with nothing but these tears and this pain'\], 'verse_two': ["We said forever,   │
│ but forever's just a lie", "And now I'm left to wonder why", 'You walked away, without a fight', 'Leaving me    │
│ here, to face the dark of night'\], 'outro': ["And I'll keep on fading, into the shadows of my mind", 'Where    │
│ love and loss entwine, and my heart will forever be left behind', "But maybe someday, I'll learn to let go",    │
│ 'And find my way back home, where love will set me free'\]}                                                     │
│     Use all the tools you are provided with. Also, analyze the predominant emotion of each stanza. Repor your   │
│ findings in a written summary.                                                                                  │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  stanzas = {                                                                                                      
      'title': 'Fading Away',                                                                                      
      'intro': ['In the silence, I hear your voice', 'A whispered reminder of our choice', 'To let go, to move     
  on, to lose', 'The love we had, the memories we chose'],                                                         
      'verse_one': ["We said forever, but forever's come and gone", "And now I'm left to face this empty dawn",    
  'Your smile, a distant memory, a fleeting thought', "I'm searching for a way to heal my heart, to make it        
  stop"],                                                                                                          
      'pre_chorus': ['But the pain remains, like an open sore', 'A constant reminder of what we had before', "I'm  
  trying to move on, but I'm stuck in this place", "Where love and loss collide, and my heart can't escape"],      
      'chorus': ["And I'm fading away, like a ghost in the night", "My heart is breaking, but it's not feeling     
  right", "I'm trying to hold on, but you're slipping through my hands", 'Leaving me with nothing but these tears  
  and this pain'],                                                                                                 
      'verse_two': ["We said forever, but forever's just a lie", "And now I'm left to wonder why", 'You walked     
  away, without a fight', 'Leaving me here, to face the dark of night'],                                           
      'outro': ['And I'll keep on fading, into the shadows of my mind', 'Where love and loss entwine, and my       
  heart will forever be left behind', "But maybe someday, I'll learn to let go", 'And find my way back home,       
  where love will set me free']                                                                                    
  }                                                                                                                
                                                                                                                   
  # Extract keywords for each stanza                                                                               
  stanzas_with_keywords = {}                                                                                       
                                                                                                                   
  for key in stanzas:                                                                                              
      if key == 'title':                                                                                           
          continue  # Skip title and intro which are not stanzas                                                   
      else:                                                                                                        
          keywords = extract_keywords(stanza=stanzas[key], top_k=10)                                               
          stanzas_with_keywords[key] = keywords                                                                    
                                                                                                                   
  print[38;2;248;248;242;48;2;3

Code parsing failed on line 8 due to: SyntaxError
    'outro': ['And I'll keep on fading, into the shadows of my mind', 'Where love and loss entwine, and my heart 
will forever be left behind', "But maybe someday, I'll learn to let go", 'And find my way back home, where love 
will set me free'\]                                                                                                
^
Error: unterminated string literal (detected at line 8) (<unknown>, line 8)

[Step 1: Duration 382.55 seconds| Input tokens: 2,511 | Output tokens: 502]

...

[Step 7: Duration 183.56 seconds| Input tokens: 38,123 | Output tokens: 2,270]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            Thought: This task has been completed and summarized without needing any code. I've provided the 
emotional summaries for each stanza based on keywords from the stanzas.

Summary:

1. Title ('Fading Away'): Love and reminiscence of past relationships.
2. Intro: Resignation towards choices or dealing with loss.
3. Verse One: Mixed feelings about commitment vs. time passing.
4. Pre-Chorus: A blend of love, longing, and sadness linked to loss.
5. Chorus: Sadness due to separation or loss resulting in heartbreak.
6. Verse Two: Sadness stemming from loneliness and regret.
7. Outro: Hints at unresolved emotions with lingering love setting free from pain.

These summaries outline the emotional tone of each stanza based on their content, reflecting themes such as 
relationships, loss, commitment, longing, and sadness.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 8: Duration 136.35 seconds| Input tokens: 46,299 | Output tokens: 2,446]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from music21 import stream, note, meter, key, stream, chord
from smolagents.tools import tool
import uuid

scores = {}

@tool
def create_score(key_signature: str = "C", time_signature: str = "4/4") -> str:
    """
    Creates a new score with specified key and time signature. Returns the score ID.

    Args:
        key_signature: key signature for the score, like C, D, E.
        time_signature: time signature for the score, like 4/4, 3/4, 2/4.
    """
    s = stream.Score()
    p = stream.Part()
    ks = key.Key(key_signature)
    ts = meter.TimeSignature(time_signature)
    p.append(ks)
    p.append(ts)
    s.append(p)

    score_id = str(uuid.uuid4())
    scores[score_id] = {"score": s, "part": p}
    print(f"New score created with id {score_id}")

    return score_id

@tool
def add_chord_to_score(score_id: str, pitches: list[str], duration: str = "quarter") -> None:
    """
    Adds a chord to the specified score.

    Args:
        score_id: ID of the score
        pitches: list of pitches, e.g., ["C4", "E4", "G4"]
        duration: duration of the chord ("whole", "half", "quarter", "eighth", "16th", etc.)
    """
    if score_id not in scores:
        return f"Error: id {score_id} not found."
    
    c = chord.Chord(pitches)
    c.duration.type = duration
    scores[score_id]["part"].append(c)
    print(f"Added chord {pitches} ({duration}) to score {score_id}.")

@tool
def add_lyrics_to_score(score_id: str, lyrics: list[str]) -> str:
    """
    Adds lyrics to the notes of the specified score.

    Args:
        score_id: ID of the score
        lyrics: list of lyrics (e.g., ["Hel-", "lo", "world"])

    Notes:
        Lyrics are assigned sequentially to each note or chord in the part.
    """
    if score_id not in scores:
        return f"Error: id {score_id} not found."
    
    part = scores[score_id]["part"]
    notes_and_chords = [n for n in part.recurse().notes]  # includes both notes and chords
    
    if len(lyrics) > len(notes_and_chords):
        return f"Error: More lyrics than notes/chords ({len(lyrics)} > {len(notes_and_chords)})."
    
    for i, lyric in enumerate(lyrics):
        notes_and_chords[i].lyric = lyric

    return f"Added {len(lyrics)} lyrics to score {score_id}."


@tool
def add_note_to_score(score_id: str, pitch: str, duration: str = "quarter") -> None:
    """
    Adds a note to the specified score.

    Args:
        score_id: id of the score
        pitch: pitch of the note, like "C4", "D5", "B3"
        duration: duration of the note ("whole", "half", "quarter", "eighth", "16th", "32nd", "64th")
    """
    if score_id not in scores:
        return f"Error: id {score_id} not found."
    
    n = note.Note(pitch)
    n.duration.type = duration
    scores[score_id]["part"].append(n)
    print(f"Added {pitch} ({duration}) to score {score_id}.")


def get_lyrics() -> str:
    """
    Gets the lyrics of the song.
    """
    sections = [
        'intro', 'verse_one', 'pre_chorus', 'chorus', 'verse_two', 'pre_chorus', 'chorus', 'outro'
    ]

    full_lyrics = ''

    for section in sections:
        full_lyrics += '(' + section + ')\n'
        for l in lyrics[section]:
            full_lyrics += l
            full_lyrics += '\n'
        full_lyrics += '\n'

    return full_lyrics


def export_score(score_id: str, format: str = "musicxml") -> None:
    """
    Exports the score to a file and returns the file path.
    """
    if score_id not in scores:
        return f"Error: score ID {score_id} not found."

    path = f"{score_id}.{format}"
    scores[score_id]["score"].write(fmt=format, fp=path)
    print(f"Score exported to: {path}")


agent = CodeAgent(
    tools=[create_score, add_note_to_score, add_chord_to_score],
    model=smolagents_model
)

score_id = agent.run('''
    You are an experienced composer. Consider a song with the following requisites:
    - Style: {style}
    - Prompt: {user_prompt}
    - Tone: {tone}
    - Lyrics: {lyrics}
    First, create a score in C major and in a time signature of your choice.
    Then, create the chords of the song according to the lyrics.
'''.format(
        style=dict_prompt['style'],
        user_prompt=dict_prompt['user_prompt'],
        tone=dict_prompt['tone'],
        lyrics=get_lyrics()
    )
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an experienced composer. Consider a song with the following requisites:                                 │
│     - Style: Pop                                                                                                │
│     - Prompt: A song about love and resilience                                                                  │
│     - Tone: Sad                                                                                                 │
│     - Chorus: ["And I'm fading away, like a ghost in the night", "My heart is breaking, but it's not feeling    │
│ right", "I'm trying to hold on, but you're slipping through my hands", 'Leaving me with nothing but these tears │
│ and this pain'\]                                                                                                │
│     First, create a score in C major and in a time signature of your choice.                                    │
│     Then, create a simple but captvating melody for the chorus of the song, according to the lyrics.            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  score_id = create_score('C', '4/4')                                                                              
  print(f"Score ID created: {score_id}")                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

New score created with id 0bdf0edd-bcfd-4cea-bf90-382066f337cf


Execution logs:
Score ID created: 0bdf0edd-bcfd-4cea-bf90-382066f337cf

Out: None

...

[Step 5: Duration 150.54 seconds| Input tokens: 14,638 | Output tokens: 830]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(score_id)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 0bdf0edd-bcfd-4cea-bf90-382066f337cf

[Step 6: Duration 44.80 seconds| Input tokens: 18,938 | Output tokens: 880]

In [ ]:
export_score('0bdf0edd-bcfd-4cea-bf90-382066f337cf')

Score exported to: 0bdf0edd-bcfd-4cea-bf90-382066f337cf.musicxml


: 